## Get Data

#### Download all match data from static dataset

In [1]:
import requests

In [72]:
def gen_static_url(num):
    url = 'https://s3-us-west-1.amazonaws.com/riot-api/seed_data/matches%d.json' % num
    return url

In [78]:
import time

In [91]:
matches = []
for num in range(1,11):
    url = gen_static_url(num)
    print "Fetching %s ..." % url
    r = requests.get(url)
    try:
        data = r.json()
        matches += data['matches']
        print "Done!"
    except:
        print "Error! Skipping."
    time.sleep(1)
print "All done!"

Fetching https://s3-us-west-1.amazonaws.com/riot-api/seed_data/matches1.json ...
Done!
Fetching https://s3-us-west-1.amazonaws.com/riot-api/seed_data/matches2.json ...
Done!
Fetching https://s3-us-west-1.amazonaws.com/riot-api/seed_data/matches3.json ...
Error! Skipping.
Fetching https://s3-us-west-1.amazonaws.com/riot-api/seed_data/matches4.json ...
Done!
Fetching https://s3-us-west-1.amazonaws.com/riot-api/seed_data/matches5.json ...
Done!
Fetching https://s3-us-west-1.amazonaws.com/riot-api/seed_data/matches6.json ...
Done!
Fetching https://s3-us-west-1.amazonaws.com/riot-api/seed_data/matches7.json ...
Done!
Fetching https://s3-us-west-1.amazonaws.com/riot-api/seed_data/matches8.json ...
Done!
Fetching https://s3-us-west-1.amazonaws.com/riot-api/seed_data/matches9.json ...
Done!
Fetching https://s3-us-west-1.amazonaws.com/riot-api/seed_data/matches10.json ...
Done!
All done!


In [102]:
len(matches)

900

#### Save all match data

In [96]:
import pickle

In [97]:
with open('matches.data', 'w') as f:
    pickle.dump(matches, f)

#### Get match data for summoners

In [235]:
import riot_games_api
reload(riot_games_api)

rg = riot_games_api.RiotGames('riot_games_api.key')

In [236]:
summonerId = 69570294

In [242]:
matchIds = rg.get_matchIds_by_summoner(summonerId)

print len(matchIds)

summonerIds = []

matchId = matchIds[0]
summonerIds += rg.get_summonerIds_by_match(matchId)

print summonerIds

Retrying in 1.00 seconds
Retrying in 2.00 seconds
Retrying in 2.00 seconds
Retrying in 2.00 seconds
Success!
173
Success!
[35549346, 31638910, 30475433, 43248772, 35736110, 29196225, 34964968, 21454109, 69570294, 64874116]


In [223]:
summonerId = 64874116
matchIds = rg.get_matchIds_by_summoner(summonerId)

print len(matchIds)

summonerIds = []

matchId = matchIds[1]
summonerIds += rg.get_summonerIds_by_match(matchId)

print summonerIds

Success!
362
Success!
[35006213, 64874116, 65621417, 50622190, 60838240, 39721094, 64870532, 53064132, 62789442, 40584540]


## Analysis

In [98]:
match = matches[0]

In [99]:
match.keys()

[u'queueType',
 u'matchVersion',
 u'platformId',
 u'season',
 u'region',
 u'matchId',
 u'mapId',
 u'matchCreation',
 u'teams',
 u'participants',
 u'matchMode',
 u'matchDuration',
 u'timeline',
 u'participantIdentities',
 u'matchType']

In [100]:
regions = set()
for match in matches:
    regions.add(match['region'])

In [101]:
regions

{u'NA'}

In [24]:
import numpy as np

In [23]:
num_teams = np.array()
for match in matchs
len(match['teams'])

2

In [18]:
len(match['participants'])

10

In [47]:
len(match['participantIdentities'])

10

In [71]:
match['participantIdentities'][0]

{u'participantId': 1,
 u'player': {u'matchHistoryUri': u'/v1/stats/player_history/NA1/229811424',
  u'profileIcon': 909,
  u'summonerId': 69570294,
  u'summonerName': u'Your Grammy'}}

In [62]:
match['participantIdentities'][0]['player']['summonerId']

69570294

In [103]:
summonerIds = set()
for match in matches:
    for p in match['participantIdentities']:
        summonerIds.add(p['player']['summonerId'])

In [104]:
len(summonerIds)

9000